In [ ]:
# Importar librerías
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

print(f"TensorFlow version: {tf.__version__}")
print("GPU disponible:", tf.config.list_physical_devices('GPU'))
print("Entorno de deep learning configurado")